In [ ]:
# code for loading in data set from:
# https://pythonbasics.org/how-to-load-machine-learning-data-in-python/
import numpy
filename = 'Data/CS170_Small_Data__82.txt'
raw_data = open(filename, 'rt')
data = numpy.loadtxt(raw_data)
print(data.shape)
print(data)